In [ ]:
from train import train_setup

In [2]:
trainer = train_setup(r'./train_config.yaml')

100%|██████████| 1000/1000 [00:00<00:00, 12859.45it/s]
/home/octopus/Documents/SberBank/TestTask/.venv/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
# set(trainer.train_dataset.data_names).intersection(set(trainer.eval_dataset.data_names))

In [ ]:
trainer.train()

In [3]:
trainer.evaluate()

  0%|          | 0/19 [00:00<?, ?it/s]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: equilibriumivan. Use `wandb login --relogin` to force relogin


{'eval_loss': 0.35294732451438904,
 'eval_dice_group_0': 0.9395546913146973,
 'eval_iou_group_0': 0.8869707584381104,
 'eval_dice_group_1': 0.8558213114738464,
 'eval_iou_group_1': 0.7488468289375305,
 'eval_dice_group_2': 0.7613247036933899,
 'eval_iou_group_2': 0.6154512166976929,
 'eval_dice_group_3': 0.6396359801292419,
 'eval_iou_group_3': 0.4726933538913727,
 'eval_dice_group_4': 0.4277057647705078,
 'eval_iou_group_4': 0.2721239924430847,
 'eval_dice': 0.8481459617614746,
 'eval_iou': 0.7533489465713501,
 'eval_runtime': 193.1787,
 'eval_samples_per_second': 0.776,
 'eval_steps_per_second': 0.098}

wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
print(trainer.train_dataset.dataset[0]['input'].shape)
trainer.train_dataset.dataset[0]['target'].shape

In [ ]:
print(trainer.eval_dataset[0]['input'].shape)
trainer.eval_dataset[0]['target'].shape

In [ ]:
trainer.train_dataset.dataset.augmentator()